## Dantar Alejandro Ortiz Vega 6-D ##
## Jose Ramon Preciado Torres 6-D ##

Ingenieria en Computacion Inteligente

Materia: Agentes Inteligentes

Bueno para esta 2da y ultima parte del proyecto ascensor,se mejoro la estructura del codigo adaptandolo a la libreta de jupyter y claro

poniendo lo que hacia falta con respecto a lo que era el algoritmo RL y el Q-learning y bueno nosotros implementamos un ambiente en el que

se nos mostraran las solicitudes atendidas tanto internas como externas viendo el comportamiento del algoritmo desarrollado

In [ ]:
import numpy as np
import time
import random

: 

In [62]:
# Definimos la clase del ascensor
class Ascensor:
    def __init__(self):
        self.lugar = 1  # Piso inicial
        self.direccion = 0  # Detenido
        self.movimiento = 0  # Detenido
        self.solicitud_interna = [0, 0, 0, 0, 0]  # No hay solicitudes internas
        self.solicitud_externa = [0, 0, 0, 0, 0]  # No hay solicitudes externas
        self.tiempo_espera_externa = [0, 0, 0, 0, 0]  # Tiempo de espera de solicitudes externas
        self.puertas_abiertas = False  # Estado inicial de las puertas

    def mover_arriba(self):
        if self.lugar < 5:
            self.lugar += 1
            self.movimiento = 1
            self.direccion = 1

    def mover_abajo(self):
        if self.lugar > 1:
            self.lugar -= 1
            self.movimiento = 1
            self.direccion = -1

    def parar(self):
        self.movimiento = 0

    def abrir_puertas(self):
        # Solo abre las puertas si hay solicitudes internas o externas en el piso actual
        if self.solicitud_interna[self.lugar - 1] == 1 or self.solicitud_externa[self.lugar - 1] != 0:
            print(f"Abrir puertas en el piso {self.lugar}")
            self.parar()  # El ascensor debe detenerse
            self.puertas_abiertas = True
            # Simulamos el tiempo de abrir puertas
            time.sleep(1)  # 1 segundo para abrir puertas (ajustar según necesidad)

    def cerrar_puertas(self):
        if self.puertas_abiertas:
            print(f"Cerrar puertas en el piso {self.lugar}")
            self.puertas_abiertas = False
            # Simulamos el tiempo de cerrar puertas
            time.sleep(1)  # 1 segundo para cerrar puertas (ajustar según necesidad)
            # Actualizar el estado del ascensor después de cerrar las puertas
            if any(self.solicitud_interna) or any(self.solicitud_externa):
                if self.direccion == 1 and self.lugar < 5:
                    self.mover_arriba()
                elif self.direccion == -1 and self.lugar > 1:
                    self.mover_abajo()
                else:
                    self.parar()
            else:
                self.parar()

    def actualizar_estado(self, accion):
        if accion == 0:
            self.mover_arriba()
        elif accion == 1:
            self.mover_abajo()
        elif accion == 2:
            self.parar()
        elif accion == 3:
            self.abrir_puertas()
        elif accion == 4:
            self.cerrar_puertas()

In [63]:
# Definimos el ambiente de aprendizaje
class Ambiente:
    def __init__(self):
        self.ascensor = Ascensor()
        self.q_table = np.zeros((5, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 5, 5, 5, 5, 5, 5))  # Inicializamos la Q-Table con ceros

    def obtener_estado(self):
        estado = [self.ascensor.lugar - 1, 
                  self.ascensor.direccion, 
                  self.ascensor.movimiento] + \
                 self.ascensor.solicitud_interna + \
                 self.ascensor.solicitud_externa
        return tuple(estado)

    def ejecutar_accion(self, accion):
        self.ascensor.actualizar_estado(accion)

    def obtener_recompensa(self, accion):
        recompensa = 0

        # Recompensa por atender solicitudes internas
        for i in range(5):
            if self.ascensor.solicitud_interna[i] == 1 and self.ascensor.lugar == i + 1:
                recompensa += 10
                self.ascensor.solicitud_interna[i] = 0

        # Recompensa por atender solicitudes externas
        for i in range(5):
            if self.ascensor.solicitud_externa[i] != 0 and self.ascensor.lugar == i + 1:
                recompensa += 15
                self.ascensor.solicitud_externa[i] = 0
                self.ascensor.tiempo_espera_externa[i] = 0

        # Penalización por tiempo de espera de solicitudes externas
        for i in range(5):
            if self.ascensor.solicitud_externa[i] != 0:
                self.ascensor.tiempo_espera_externa[i] += 1
                recompensa -= self.ascensor.tiempo_espera_externa[i]

        # Penalización por cambio de dirección innecesario
        if accion == 0 and self.ascensor.direccion == -1 and any(self.ascensor.solicitud_interna):
            recompensa -= 5
        if accion == 1 and self.ascensor.direccion == 1 and any(self.ascensor.solicitud_interna):
            recompensa -= 5

        return recompensa

    def step(self, accion):
        estado_anterior = self.obtener_estado()
        self.ejecutar_accion(accion)
        recompensa = self.obtener_recompensa(accion)
        estado_nuevo = self.obtener_estado()
        return estado_anterior, accion, recompensa, estado_nuevo

    def reset(self):
        self.ascensor = Ascensor()
        return self.obtener_estado()

In [64]:
# Parámetros de Q-Learning
alpha = 0.1  # Tasa de aprendizaje
gamma = 0.9  # Factor de descuento
epsilon = 1.0  # Tasa de exploración inicial
epsilon_min = 0.01  # Tasa de exploración mínima
epsilon_decay = 0.995  # Decadencia de la tasa de exploración

In [65]:
# Función para seleccionar una acción usando la política epsilon-greedy
def seleccionar_accion(estado):
    if np.random.rand() <= epsilon:
        return random.choice([0, 1, 2, 3, 4])
    q_values = ambiente.q_table[estado]
    return np.argmax(q_values)

In [ ]:
# Inicializar el ambiente
ambiente = Ambiente()

# Función para imprimir el estado del ascensor
def imprimir_estado(ascensor):
    print(f"Estado del ascensor: Piso {ascensor.lugar}, Dirección {ascensor.direccion}, Movimiento {ascensor.movimiento}")
    print(f"Solicitudes internas: {ascensor.solicitud_interna}")
    print(f"Solicitudes externas: {ascensor.solicitud_externa}")

# Entrenamiento de Q-Learning
episodios = 1000
for episodio in range(episodios):
    estado = ambiente.reset()
    done = False
    imprimir_estado(ambiente.ascensor)

    while not done:
        accion = seleccionar_accion(estado)
        acciones_str = ["mover_arriba", "mover_abajo", "parar", "abrir_puertas", "cerrar_puertas"]
        print(f"Acción: {acciones_str[accion]}")
        estado_anterior, accion, recompensa, estado_nuevo = ambiente.step(accion)
        imprimir_estado(ambiente.ascensor)

        best_next_action = np.argmax(ambiente.q_table[estado_nuevo])
        td_target = recompensa + gamma * ambiente.q_table[estado_nuevo][best_next_action]
        td_error = td_target - ambiente.q_table[estado_anterior][accion]
        ambiente.q_table[estado_anterior][accion] += alpha * td_error

        estado = estado_nuevo

        if sum(ambiente.ascensor.solicitud_externa) == 0 and sum(ambiente.ascensor.solicitud_interna) == 0:
            done = True

    if epsilon > epsilon_min:
        epsilon *= epsilon_decay

print("Entrenamiento completado.")